In [1]:
%load_ext nbextension
from datetime import datetime

import numpy as np
import pandas as pd
import xarray as xr
import dask
import dask.array as da
import dask.dataframe as dd


In [31]:
from __future__ import annotations
from typing import (
    Generic,
    Protocol,
    ParamSpec,
    Callable,
    NewType,
    TypeVar,
    Tuple,
    Sequence,
)
from typing_extensions import TypeVarTuple
from numpy.typing import NDArray



In [21]:
from collections import namedtuple
from typing import Callable, Iterable, NamedTuple, get_args, get_origin, get_type_hints

from gswp.typing import Self


def _get_members(cls: type):
    def callback(func: Callable):
        return lambda *args, **kwargs: func(cls, *args, **kwargs)

    for key, value in vars(cls).items():
        if key.startswith("_"):
            continue
        if isinstance(value, Callable):
            value = callback(value)
        yield key, value


def _repr_members(cls: type) -> Iterable:
    for key, value in vars(cls).items():
        if key.startswith("_"):
            continue
        if isinstance(value, Callable):
            hints = get_type_hints(value)

            return_value = hints.pop("return", None)
            args = ", ".join(f"{k}:{v.__name__}" for k, v in hints.items())
            yield f'{key}({args}) -> {return_value.__name__ if return_value else "Any"}'

        else:
            yield f"{key}:{type(value).__name__} = {value}"


def frozen_singleton(cls: type[Self]) -> Self:
    members = dict(_get_members(cls))
    class _SINGLETON(namedtuple(cls.__name__, members.keys())):
        __name__ = cls.__name__

        def __repr__(self) -> str:
            props = "\n  ".join(_repr_members(cls))
            return f"FROZEN[{cls.__name__}]:\n  {props}"

    return _SINGLETON(**members)


@frozen_singleton
class FS:
    a = 1, 2, 3, 5, 5, 6
    b = 112323
    c = "World"

    def one(self) -> None:
        ...

    def two(self, hello: str = None) -> int:
        ...

    def three(self):
        ...

    @classmethod
    def __a(self): 
        return 1
        
    def get_sunder_a(self):
        return self.__a()


print(FS.get_sunder_a())
FS

1


FROZEN[FS]:
  a:tuple = (1, 2, 3, 5, 5, 6)
  b:int = 112323
  c:str = World
  one() -> NoneType
  two(hello:Optional) -> int
  three() -> Any
  get_sunder_a() -> Any